In [ ]:
import pandas as pd
from tqdm import tqdm
from pymongo import MongoClient
import py_vncorenlp
import re
from collections import Counter
import os
from collections import defaultdict
import numpy as np
from math import log
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
import math
import re
import unicodedata

uri = ""

client = MongoClient(uri)

db = client["nlp"]      
article_collection = db["article"]      

In [2]:
df = pd.read_json('all_articles_v2.json')

In [3]:
df = df.drop_duplicates(subset=['id']) 
df = df[df['content'].str.len()>0]

In [4]:
def fix_spacing(text):
    text = re.sub(r'([.,!?;:])(?=\S)', r'\1 ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def is_writer_signature(sentence):
    sentence = sentence.strip().rstrip('.')
    
    if len(sentence.split()) <= 3 and sentence == sentence.title():
        return True
    
    if re.fullmatch(r'[A-Z]\.?', sentence) or re.fullmatch(r'[A-Z][a-z]+(\s[A-Z][a-z]+)?', sentence):
        return True

    if "Thực hiện" in sentence or sentence.upper() == sentence:
        return True

    return False

def remove_writer_name(text):
    sentences = [s.strip() for s in text.strip().split('.') if s.strip()]
    if not sentences:
        return text
    last = sentences[-1]
    if is_writer_signature(last):
        return '. '.join(sentences[:-1]) + '.' if len(sentences) > 1 else ''
    return text


In [5]:
# clear stuff like \n \xa0... from df['content]
df['content'] = df['content'].apply(lambda x: re.sub(r'[\n\r\t\xa0\u200b\u202f]+', ' ', str(x)).strip())
# fix spacing
df['content'] = df['content'].apply(fix_spacing)
# remove writer name
df['content'] = df['content'].apply(remove_writer_name)

In [6]:
df[df['content'].str.len() > 0].shape

(6347, 13)

In [7]:
df.iloc[0]['content']

'STO - Chiều ngày 28/5, tại UBND thành phố Sóc Trăng (Sóc Trăng), Thường trực HĐND thành phố Sóc Trăng khóa XII, nhiệm kỳ 2021 - 2026 tổ chức kỳ họp thứ 23 (chuyên đề) quyết định các nội dung thuộc thẩm quyền của HĐND thành phố. Đồng chí Nguyễn Thị Kiều Linh - Phó Bí thư Thường trực Thành ủy, Chủ tịch HĐND thành phố và đồng chí Lê Thanh Hùng - Phó Chủ tịch HĐND thành phố Sóc Trăng chủ tọa kỳ họp. Tại kỳ họp, UBND thành phố thông qua các tờ trình và dự thảo nghị quyết về phê duyệt quyết toán thu chi ngân sách năm 2024; điều chỉnh bổ sung kế hoạch đầu tư công trung hạn giai đoạn 2021 - 2025 ngân sách địa phương thành phố Sóc Trăng; điều chỉnh, bổ sung kế hoạch vốn đầu tư năm 2025 thuộc nguồn vốn ngân sách địa phương (lần 1) theo quy định của pháp luật. Trên cơ sở thảo luận tập trung, dân chủ, xem xét các tờ trình của UBND thành phố trình tại kỳ họp, HĐND thành phố đã nhất trí thông qua 3 nghị quyết. Phát biểu tại kỳ họp, đồng chí Nguyễn Thị Kiều Linh nhấn mạnh, để thực hiện có hiệu qu

In [8]:
df.head(2)

,subTitle,cateId,publishDate,pageUrl,id,avatar,avatarApp,title,lead,huyen,linhVuc,author,content
0,,8a108e1691a109da0191a10a966f0008,2025-05-28 16:11:31.0,/chinh-tri/202505/thong-qua-3-nghi-quyet-tai-k...,8e61a0b496c358a4019715ea84a47263,/8e61a0b4907d319401909010f0cf012c/052025/bieu_...,/file/8e61a0b4907d319401909010f0cf012c/052025/...,Thông qua 3 nghị quyết tại kỳ họp thứ 23 HĐND ...,"STO - Chiều ngày 28/5, tại UBND thành phố Sóc ...",Thành phố Sóc Trăng,chinh tri,HUỲNH NHƯ,"STO - Chiều ngày 28/5, tại UBND thành phố Sóc ..."
1,Ủy ban Mặt trận Tổ quốc Việt Nam thành phố Sóc...,8a108e1691a109da0191a10a9803000c,2025-05-28 12:43:35.0,/dai-doan-ket/202505/uy-ban-mat-tran-to-quoc-v...,8e61a0b496c358a401970a6a66aa00b9,/8e61a0b4907d319401909010f0cf012c/052025/anh_2...,/file/8e61a0b4907d319401909010f0cf012c/052025/...,Lan tỏa tinh thần đại đoàn kết,STO - Hưởng ứng mạnh mẽ phong trào thi đua “Ch...,Thành phố Sóc Trăng,dai doan ket,HẢI AN,STO - Hưởng ứng mạnh mẽ phong trào thi đua “Ch...


In [9]:
# py_vncorenlp.download_model(save_dir='vncorenlp')

In [10]:
original_cwd = os.getcwd()
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir=f'C:/Users/Lenovo/Desktop/work/ctu/NLP/nlp-ctu/vncorenlp')
os.chdir(original_cwd)

In [11]:
with open('vietnamese-stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = set(line.strip().lower() for line in f if line.strip())
stopwords.add('sto')
len(stopwords)    

1943

In [12]:
def is_noise_token(w):
    return re.fullmatch(r'[\W_]+', w) is not None 

def get_word_frequencies(text):
    segmented_sentences = rdrsegmenter.word_segment(text)
    all_words = []
    for sentence in segmented_sentences:
        words = sentence.split()
        
        words = [
            w for w in words 
            if w.lower() not in stopwords and not is_noise_token(w)
        ]

        words = [w.replace("_", " ") for w in words]
        all_words.extend(words)
    return dict(Counter(all_words))

In [13]:
df['word_freq'] = df['content'].apply(get_word_frequencies)

In [14]:
def compute_tf(word_freq):
    total_words = sum(word_freq.values())
    return {word: count / total_words for word, count in word_freq.items()}


In [15]:
df['TF'] = df['word_freq'].apply(compute_tf)

In [16]:
def is_valid_token(token):
    token = token.strip()

    if len(token) < 2:
        return False

    token = unicodedata.normalize('NFKC', token)

    if re.fullmatch(r"[\W_]+", token):  
        return False

    if any(unicodedata.category(c).startswith('C') for c in token):
        return False

    if token[0] in {'–', '‰', '↵', '•', '-', '+', '“', '‘'}:
        return False

    return True

def clean_tf_dict(tf_dict):
    return {
        word: freq for word, freq in tf_dict.items()
        if is_valid_token(word)
    }

In [17]:
df['TF'] = df['TF'].apply(clean_tf_dict)  

In [18]:
df.shape

(6347, 15)

In [19]:
df[df['TF'].apply(lambda x: len(x) != 0)].shape #

(6347, 15)

In [20]:
word_doc_count = defaultdict(int)

for tf_dict in df['TF']:
    for word in tf_dict.keys():
        word_doc_count[word] += 1

total_docs = len(df)

df_df = pd.DataFrame(
    {
        'word': list(word_doc_count.keys()),
        'DF': [word_doc_count[w]  for w in word_doc_count] #/total_docs
    }
)

In [21]:
df_df

,word,DF
0,Chiều,1000
1,28/5,7
2,UBND,3788
3,thành phố,1073
4,Sóc Trăng,4375
...,...,...
31457,Trần Văn Ri,1
31458,Thật tình,1
31459,chăm chăm,1
31460,êm ái,1


In [22]:
inverted_index_dict = defaultdict(dict)

for doc_id, tf_dict in zip(df['id'], df['TF']):
    for word, tf in tf_dict.items():
        inverted_index_dict[word][doc_id] = tf

df_inverted_index = pd.DataFrame(
    [{"word": word, "inverted_index": postings}
     for word, postings in inverted_index_dict.items()]
)

In [23]:
df_inverted_index

,word,inverted_index
0,Chiều,{'8e61a0b496c358a4019715ea84a47263': 0.0060975...
1,28/5,{'8e61a0b496c358a4019715ea84a47263': 0.0060975...
2,UBND,{'8e61a0b496c358a4019715ea84a47263': 0.0243902...
3,thành phố,{'8e61a0b496c358a4019715ea84a47263': 0.0792682...
4,Sóc Trăng,{'8e61a0b496c358a4019715ea84a47263': 0.0304878...
...,...,...
31457,Trần Văn Ri,{'8e61a0b491a87e280191a88521c11ca8': 0.001953125}
31458,Thật tình,{'8e61a0b491a87e280191a88521c11ca8': 0.001953125}
31459,chăm chăm,{'8e61a0b491a87e280191a88521c11ca8': 0.001953125}
31460,êm ái,{'8e61a0b491a87e280191a88521c11ca8': 0.001953125}


In [24]:

doc_total_words = {
    doc_id: sum(word_freq.values())
    for doc_id, word_freq in zip(df['id'], df['word_freq'])
}

rows = []

for doc_id, word_freq in zip(df['id'], df['word_freq']):
    total_words = doc_total_words[doc_id]
    for word, count in word_freq.items():
        rows.append({
            "word": word,
            "articleId": doc_id,
            "tf": count,
            "frequency": count / total_words
        })
        
df_tf_full = pd.DataFrame(rows)

df_tf_full = df_tf_full.pivot_table(
    index='articleId',
    columns='word',
    values='frequency',
    fill_value=0
)
df_tf_full.columns.name = None
df_tf_full.index.name = None

N = df_tf_full.shape[0]

df_counts = (df_tf_full > 0).sum(axis=0)

idf = np.log(N / df_counts)

df_tf_idf_full = df_tf_full * idf

In [25]:
df_tf_idf_full.head(2)

,VĂN,VĂN LONG,(4/1996,(5/1975,(7/2005,(9/1986,+12,+61,", son",", thống",...,ứng xử,ực,ựu,ỳ,ỷ,ỷ lại,ỹ,–CT,↵Cách,州
8e61a0b491a254630191ac801c4a2bcc,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8e61a0b491a254630191ac9c6cfc2cb5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
print(df_tf_idf_full.columns.tolist())

[' VĂN', ' VĂN LONG', '(4/1996', '(5/1975', '(7/2005', '(9/1986', '+12', '+61', ', son', ', thống', ', tiến', '-17', '-19', '-2', '-29', '-44', '-47', '-51', '-53', '-55', '-57', '-61', '0', '0%', '00', '000', '000/165', '00000', '00075', '000ha', '000kg', '000km', '000m', '000m2', '000m3', '000mm', '001', '002', '003', '004', '004ha', '005', '005ha', '006', '007', '008', '008ha', '009', '00m', '01', '01%', '01-25', '01/134', '01/2013/QĐST-DS', '01/2017/CT-TA', '01/2020/TT-BTP', '01/2020/TT-NHNN;', '01/2020/TTLT-VKSNDTC-TANDTC-BCA-BQP-BNN&PTNT-BTC-BTP', '010', '0108', '011', '012', '013', '014', '015', '016', '017', '018', '01840', '018ha', '019', '01SL', '01ha', '02', '02%', '02/2013/QĐST-DS', '02/2013/TT-BVHTTDL', '02/2014/QCLN/CTHADS-CA-TAND-VKSND', '02/2020/PL-UBTVQH14', '02/2020/UBTVQH', '02/2023/NQ-HĐND', '02/7', '020', '020ha', '020m', '021', '022', '022m', '023', '024', '025', '026', '027', '028', '029', '0299', '02992200113', '02993', '02993501113', '02x40MVA', '03', '03%', '0

In [27]:
clean_columns = [col for col in df_tf_idf_full.columns if is_valid_token(col)]
df_tf_idf_full = df_tf_idf_full[clean_columns]

In [28]:
df_tf_idf_full.shape

(6347, 31462)

In [29]:
print(df_tf_idf_full.columns.tolist())

[' VĂN', ' VĂN LONG', '(4/1996', '(5/1975', '(7/2005', '(9/1986', ', son', ', thống', ', tiến', '0%', '00', '000', '000/165', '00000', '00075', '000ha', '000kg', '000km', '000m', '000m2', '000m3', '000mm', '001', '002', '003', '004', '004ha', '005', '005ha', '006', '007', '008', '008ha', '009', '00m', '01', '01%', '01-25', '01/134', '01/2013/QĐST-DS', '01/2017/CT-TA', '01/2020/TT-BTP', '01/2020/TT-NHNN;', '01/2020/TTLT-VKSNDTC-TANDTC-BCA-BQP-BNN&PTNT-BTC-BTP', '010', '0108', '011', '012', '013', '014', '015', '016', '017', '018', '01840', '018ha', '019', '01SL', '01ha', '02', '02%', '02/2013/QĐST-DS', '02/2013/TT-BVHTTDL', '02/2014/QCLN/CTHADS-CA-TAND-VKSND', '02/2020/PL-UBTVQH14', '02/2020/UBTVQH', '02/2023/NQ-HĐND', '02/7', '020', '020ha', '020m', '021', '022', '022m', '023', '024', '025', '026', '027', '028', '029', '0299', '02992200113', '02993', '02993501113', '02x40MVA', '03', '03%', '03%/2%', '03/2', '03/2015/NQ-HĐND', '03/2018/HSST', '03/2018/NQ-HĐND', '03/2019/NQ-HĐND', '03/20

In [30]:
def rank_documents_by_query(query, tf_idf, tokenizer, stopwords):
    # tokenize query
    segmented = tokenizer.word_segment(query)
    query_tokens = []
    for sentence in segmented:
        words = sentence.split()
        words = [w.replace("_", " ") for w in words]
        words = [w.lower() for w in words if w.lower() not in stopwords]
        query_tokens.extend(words)

    word_list = tf_idf.columns
    query_vector = np.zeros(len(word_list))

    word_counts = {word: query_tokens.count(word) for word in set(query_tokens)}

    total_terms = sum(word_counts.values())
    if total_terms == 0:
        return []

    for i, term in enumerate(word_list):
        if term in word_counts:
            query_vector[i] = word_counts[term] / total_terms 

    cosin_sim = cosine_similarity([query_vector], tf_idf.values)[0]

    article_ids = tf_idf.index.tolist()
    ranked = sorted(zip(article_ids, cosin_sim), key=lambda x: x[1], reverse=True)
    
    return ranked

In [31]:
def CosineScore(query, tf_idf, tokenizer, stopwords):
    # Step 1: float Scores[N] = 0
    Scores = {doc_id: 0.0 for doc_id in tf_idf.index}

    # Step 2: float Length[N]
    Length = {}
    for doc_id, row in tf_idf.iterrows():
        Length[doc_id] = math.sqrt(sum(v ** 2 for v in row.values)) or 1e-9

    # Step 3: for each query term t
    segmented = tokenizer.word_segment(query)
    query_tokens = []
    for sentence in segmented:
        words = sentence.split()
        words = [w.replace("_", " ") for w in words]
        words = [w.lower() for w in words if w.lower() not in stopwords]
        query_tokens.extend(words)

    # Calculate tf for query terms
    tf_query = {}
    for t in query_tokens:
        tf_query[t] = tf_query.get(t, 0) + 1
    total_terms_in_query = sum(tf_query.values())

    # Step 4: calculate w_{t,q} and fetch postings list for t
    for t in tf_query:
        if t not in tf_idf.columns:
            continue

        w_tq = tf_query[t] / total_terms_in_query  # query weight for term t

        # Step 5–6: for each (d, tf_{t,d}) in postings list, update Scores[d]
        for d, w_td in tf_idf[t].items():
            Scores[d] += w_td * w_tq

    # Step 7 is already done when we computed Length

    # Step 8–9: Normalize Scores[d] = Scores[d] / Length[d]
    for d in Scores:
        Scores[d] = Scores[d] / Length[d]

    # Step 10: return Top K components of Scores[]
    ranked_scores = sorted(Scores.items(), key=lambda x: x[1], reverse=True)
    return ranked_scores


In [ ]:
# # raw -> db mongo ✓

# result = article_collection.delete_many({})

# documents = df.to_dict(orient="records")
# article_collection.insert_many(documents)

# #tf -> db mongo ✓

# tf_collection = db["article_tf"]

# tf_collection.delete_many({})

# docs = [
#     {"articleId": row["id"], "tf": row["TF"]}
#     for _, row in df.iterrows()
# ]

# tf_collection.insert_many(docs)

# #df -> db mongo ✓

# df_collection = db["article_df"]

# df_collection.delete_many({})

# docs = [
#     {"articleId": row["word"], "df": row["DF"]}
#     for _, row in df_df.iterrows()
# ]

# df_collection.insert_many(docs)

# # inverted_index -> db mongo ✓

# inverted_index_collection = db["article_inverted_index"]

# inverted_index_collection.delete_many({})

# docs = [
#     {"word": row["word"], "inverted_index": row["inverted_index"]}
#     for _, row in df_inverted_index.iterrows()
# ]

# inverted_index_collection.insert_many(docs)

# # tf_idf -> db mongo ✓

# article_tf_idf_collection = db["article_tf_idf"]
# article_tf_idf_collection.delete_many({})

# for article_id, row in tqdm(df_tf_idf_full.iterrows(), total=len(df_tf_idf_full)):
#     tf_idf_dict = row[row != 0].to_dict()  
#     doc = {
#         "articleId": article_id,
#         "tf_idf": tf_idf_dict
#     }
#     article_tf_idf_collection.insert_one(doc)

100%|██████████| 6347/6347 [05:22<00:00, 19.70it/s]


In [33]:
search_term = "tai nạn giao thông"

In [34]:
results = rank_documents_by_query(search_term, df_tf_idf_full, rdrsegmenter, stopwords)

for articleId, score in results[:10]:
    title = df.loc[df['id'] == articleId, 'title'].values[0]
    print(f"{title} - {score:.4f}")

Tai nạn giao thông trên địa bàn huyện Thạnh Trị giảm sâu - 0.6142
Thăm nạn nhân bị tai nạn giao thông có hoàn cảnh khó khăn - 0.6066
Tập trung thực hiện các giải pháp đảm bảo trật tự an toàn giao thông trong năm 2018 - 0.5846
Tăng cường công tác bảo đảm trật tự an toàn giao thông - 0.5666
Long Phú nỗ lực kéo giảm tai nạn giao thông - 0.5164
Gần 100 đoàn viên được tuyên truyền pháp luật về trật tự an toàn giao thông - 0.4803
Nỗ lực kéo giảm tai nạn giao thông - 0.4774
TP. Sóc Trăng đẩy mạnh thực hiện đồng bộ các giải pháp đảm bảo trật tự an toàn giao thông - 0.4685
Long Phú nhân rộng mô hình "Cổng trường an toàn giao thông" - 0.4636
Tăng cường kiểm soát tốc độ phương tiện tuyến Quốc lộ Quản lộ Phụng Hiệp - 0.4417


In [35]:
results = CosineScore(search_term, df_tf_idf_full, rdrsegmenter, stopwords)

for articleId, score in results[:10]:
    title = df.loc[df['id'] == articleId, 'title'].values[0]
    print(f"{title} - {score:.4f}")

Tai nạn giao thông trên địa bàn huyện Thạnh Trị giảm sâu - 0.4343
Thăm nạn nhân bị tai nạn giao thông có hoàn cảnh khó khăn - 0.4289
Tập trung thực hiện các giải pháp đảm bảo trật tự an toàn giao thông trong năm 2018 - 0.4133
Tăng cường công tác bảo đảm trật tự an toàn giao thông - 0.4006
Long Phú nỗ lực kéo giảm tai nạn giao thông - 0.3651
Gần 100 đoàn viên được tuyên truyền pháp luật về trật tự an toàn giao thông - 0.3396
Nỗ lực kéo giảm tai nạn giao thông - 0.3376
TP. Sóc Trăng đẩy mạnh thực hiện đồng bộ các giải pháp đảm bảo trật tự an toàn giao thông - 0.3313
Long Phú nhân rộng mô hình "Cổng trường an toàn giao thông" - 0.3278
Tăng cường kiểm soát tốc độ phương tiện tuyến Quốc lộ Quản lộ Phụng Hiệp - 0.3123


In [ ]:
df_tf_idf_full.shape

(6347, 31462)

: 